#### Bibliothèques

In [ ]:
import pandas as pd
import numpy as np

#### Importation des données

In [ ]:
#Lecture de la base simplifiée
bsimple = pd.read_csv(r"C:\Users\mamad\Desktop\MRC\baseSimplifiee.csv", sep=";")

# Lecture des fichiers CSV du premier ensemble
cm1 = pd.read_csv(r"C:\Users\mamad\Desktop\MRC\ascii-data-files_nida-ctn-0001\cm.csv") 

# Lecture des fichiers CSV du second ensemble
cm2 = pd.read_csv(r"C:\Users\mamad\Desktop\MRC\ascii-data-files_nida-ctn-0002\cm.csv") 


#### Data management

In [ ]:
# Afficher les premières lignes
print(bsimple.head())

# Afficher les informations 
print(bsimple.info())

# Statistiques descriptives
print(bsimple.describe())

#### Data processing

In [ ]:
# Liste des médicaments pour l'anxiété et l'insomnie
medications_anxiety = [
    "OXAZEPAM", "PHENOBARBITOL", "HYDROXYZENE HYDROCHLORIDE", "LORAZEPAM"
]
medications_insomnia = [
    "ZOLPIDEM TARTRATE", "TRAZADONE HYDROCHLORIDE", "DOXEPIN HYDROCHLORIDE", "DIPHENHYDRAMINE HCL"
]
all_medications = medications_anxiety + medications_insomnia

# Fonction pour traiter chaque ensemble de données
def process_dataset(bsimple, cm):
    # Filtrage des médicaments dans cm
    filtered_cm = cm[
        (cm['CMTRT'].str.contains('|'.join(all_medications), case=True, na=False)) 
    ].copy()
    
    # Conversion de VISITNUM en entier pour éviter les erreurs
    filtered_cm['VISITNUM'] = pd.to_numeric(filtered_cm['VISITNUM'], errors='coerce')
    
    # Sélection des colonnes pertinentes dans bsimple pour les patients filtrés
    tab0 = bsimple[['USUBJID', 'ARM', 'AGE', 'GENDER','MARITAL', 
                    'EMPLOYMENT', 'HEIGHT', 'WEIGHT', 'METHADONE.sc', 
                    'MORPHINE.sc', 'VISITNUM']].copy()
    tab0 = tab0[tab0['USUBJID'].isin(filtered_cm['USUBJID'].unique())]
    
    # Convertir les colonnes AGE, HEIGHT, WEIGHT, et VISITNUM en entiers
    tab0['AGE'] = tab0['AGE'].apply(lambda x: int(x) if pd.notna(x) else 0)  # Remplacer NaN par 0 si nécessaire
    tab0['HEIGHT'] = tab0['HEIGHT'].apply(lambda x: int(x) if pd.notna(x) else 0)
    tab0['WEIGHT'] = tab0['WEIGHT'].apply(lambda x: int(x) if pd.notna(x) else 0)
    tab0['VISITNUM'] = tab0['VISITNUM'].apply(lambda x: int(x) if pd.notna(x) else 0)
    
    # Création de colonnes pour chaque combinaison Médicament/Visite avec un DataFrame temporaire
    medication_visit_columns = pd.DataFrame(False, index=tab0.index,
                                            columns=[f"{med}_VISIT{visit_num}" for med in all_medications for visit_num in range(1, 14)])
    
    # Ajouter les colonnes de visites au DataFrame principal
    tab0 = pd.concat([tab0, medication_visit_columns], axis=1)
    
    # Itération sur les lignes de filtered_cm pour marquer la prise des médicaments
    for index, row in filtered_cm.iterrows():
        usubjid = row['USUBJID']
        medication = row['CMTRT']
        visit_num = row['VISITNUM']
        
        if pd.notna(visit_num) and 1 <= visit_num <= 13:
            for med in all_medications:
                if med in medication:
                    tab0.loc[tab0['USUBJID'] == usubjid, f"{med}_VISIT{int(visit_num)}"] = True
    
    # Ajouter la dernière valeur de VISITNUM pour chaque patient
    last_visit = filtered_cm.groupby('USUBJID')['VISITNUM'].max().reset_index()
    last_visit.columns = ['USUBJID', 'LAST_VISITNUM']

    # Joindre la dernière valeur de VISITNUM avec la table principale
    tab0 = pd.merge(tab0, last_visit, on='USUBJID', how='left')

    return tab0

# Création des final_table_1 et final_table_2

final_table_0 = process_dataset(bsimple, cm1)
final_table_1 = process_dataset(bsimple, cm2)

# Combinaison des tables finales
final_combined_table = pd.concat([final_table_0, final_table_1], ignore_index=True).drop_duplicates()

# Affichage pour vérification
final_combined_table.head()

,USUBJID,ARM,AGE,GENDER,MARITAL,EMPLOYMENT,HEIGHT,WEIGHT,METHADONE.sc,MORPHINE.sc,...,DIPHENHYDRAMINE HCL_VISIT5,DIPHENHYDRAMINE HCL_VISIT6,DIPHENHYDRAMINE HCL_VISIT7,DIPHENHYDRAMINE HCL_VISIT8,DIPHENHYDRAMINE HCL_VISIT9,DIPHENHYDRAMINE HCL_VISIT10,DIPHENHYDRAMINE HCL_VISIT11,DIPHENHYDRAMINE HCL_VISIT12,DIPHENHYDRAMINE HCL_VISIT13,LAST_VISITNUM
0,01_000579,CLONIDINE,27,F,DIVORCED,UNEMPLOYED,67,130,NEGATIVE,POSITIVE,...,False,False,False,False,False,False,False,False,False,3
1,01_001490,BUPRENORPHINE/NALOXONE,30,M,NEVER MARRIED,UNEMPLOYED,72,155,NEGATIVE,POSITIVE,...,False,False,False,False,False,False,False,False,False,6
2,01_002199,BUPRENORPHINE/NALOXONE,20,M,NEVER MARRIED,PART TIME (REGULAR HOURS),0,0,NEGATIVE,NEGATIVE,...,False,False,False,False,False,False,False,False,False,10
3,01_003330,CLONIDINE,32,M,LEGALLY MARRIED,FULL TIME (35+ HRS/WK),70,175,NEGATIVE,NEGATIVE,...,False,False,False,False,False,False,False,False,False,6
4,01_003653,BUPRENORPHINE/NALOXONE,26,M,NEVER MARRIED,PART TIME (IRREGULAR DAYWORK),71,215,NEGATIVE,POSITIVE,...,False,False,False,False,False,False,False,False,False,9


#### Exportation du fichier 

In [34]:
# Sauvegarder dans le fichier CSV
output_path = r"C:\Users\mamad\Desktop\MRC\final_combined_table.csv"
final_combined_table.to_csv(output_path, index=False)
print(f"Le fichier CSV a été sauvegardé avec succès à l'emplacement : {output_path}")

Le fichier CSV a été sauvegardé avec succès à l'emplacement : C:\Users\mamad\Desktop\MRC\final_combined_table.csv
